# Setup

In [4]:
import helper_tools.parser as parser
import importlib
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

importlib.reload(parser)

relation_df, entity_df, docs = parser.synthie_parser("train")
entity_set = entity_df[['entity', 'entity_uri']].drop_duplicates()
predicate_set_df = relation_df[["predicate", "predicate_uri"]].drop_duplicates()

Fetching 27 files:   0%|          | 0/27 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 5991.01it/s]


Uploading Entities to Qdrant.


100%|██████████| 46/46 [00:05<00:00,  8.71it/s]


Uploading Predicates to Qdrant.


100%|██████████| 29/29 [00:03<00:00,  9.16it/s]


In [5]:
from langgraph.graph import StateGraph, START, END
from approaches.baseline.setup import cIEState, langfuse_handler
from approaches.baseline.agents.supervisor import agent as supervisor_agent
from approaches.baseline.agents.entity_extractor import agent as entity_extraction_agent
from approaches.baseline.agents.relation_extractor import agent as relation_extraction_agent
from approaches.baseline.agents.uri_detector import agent as uri_detection_agent

builder = StateGraph(cIEState)
builder.add_node("supervisor", supervisor_agent)
builder.add_node("entity_extraction_agent", entity_extraction_agent)
builder.add_node("relation_extraction_agent",relation_extraction_agent)
builder.add_node("uri_detection_agent",uri_detection_agent)

builder.add_edge(START, "supervisor")

graph = builder.compile()

In [6]:
target_doc = docs.iloc[3]
doc_id = target_doc["docid"]
text = target_doc["text"]
text

'Lambda Mensae is a star in the constellation Mensa. It was discovered by Nicolas-Louis de Lacaille, and named after Table Mountain in South Africa. Mensa shares borders with Chamaeleon and Hydrus.'

In [7]:
response_state = graph.invoke({"text": text, "messages":[], "debug": False}, config={"callbacks": [langfuse_handler], "recursion_limit": 100})

# Pretty Print Response State

In [8]:
print(f"""cIE for text: {response_state["text"]}

Messages:""")
for message in response_state["messages"]:
    print(message)

cIE for text: Lambda Mensae is a star in the constellation Mensa. It was discovered by Nicolas-Louis de Lacaille, and named after Table Mountain in South Africa. Mensa shares borders with Chamaeleon and Hydrus.

Messages:

-- Supervisor Agent --
<goto>entity_extraction_agent</goto>

-- Entity Extraction Agent --
[Lambda Mensae, star, constellation Mensa, Nicolas-Louis de Lacaille, Table Mountain, South Africa, Mensa, Chamaeleon, Hydrus]

-- Supervisor Agent --
<goto>relation_extraction_agent</goto>

-- Relation Extraction Agent --
<relation>Lambda Mensae;is in;Mensa</relation>
<relation>Lambda Mensae;discovered by;Nicolas-Louis de Lacaille</relation>
<relation>Lambda Mensae;named after;Table Mountain</relation>
<relation>Mensa;shares borders with;Chamaeleon</relation>
<relation>Mensa;shares borders with;Hydrus</relation>
<relation>Lambda Mensae;located in;constellation Mensa</relation>
<relation>Table Mountain;located in;South Africa</relation>

-- Supervisor Agent --
<goto>uri_detecti

# Evaluation

In [9]:
from helper_tools.evaluation import parse_turtle, evaluate_doc, get_uri_labels

In [10]:
import re

turtle_string = re.search(r'<ttl>(.*?)</ttl>', response_state["messages"][-1], re.DOTALL).group(1)
pred_relation_df = parse_turtle(turtle_string)
doc_relation_df = relation_df[relation_df["docid"] == doc_id][["subject_uri", "predicate_uri", "object_uri"]]
correct_relation_df = pred_relation_df.merge(doc_relation_df[["subject_uri", "predicate_uri", "object_uri"]], on=["subject_uri", "predicate_uri", "object_uri"], how="inner")

In [11]:
get_uri_labels(pred_relation_df, entity_set, predicate_set_df)

,subject_uri,predicate_uri,object_uri,subject,predicate,object
0,http://www.wikidata.org/entity/Q9289,http://www.wikidata.org/entity/P47,http://www.wikidata.org/entity/Q10457,Mensa_(constellation),shares border with,Chamaeleon
1,http://www.wikidata.org/entity/Q3304236,http://www.wikidata.org/entity/P138,http://www.wikidata.org/entity/Q213360,Lambda_Mensae,named after,Table_Mountain
2,http://www.wikidata.org/entity/Q213360,http://www.wikidata.org/entity/P131,http://www.wikidata.org/entity/Q258,Table_Mountain,located in the administrative territorial entity,Unknown
3,http://www.wikidata.org/entity/Q3304236,http://www.wikidata.org/entity/P61,http://www.wikidata.org/entity/Q202703,Lambda_Mensae,discoverer or inventor,Nicolas-Louis_de_Lacaille
4,http://www.wikidata.org/entity/Q9289,http://www.wikidata.org/entity/P47,http://www.wikidata.org/entity/Q10416,Mensa_(constellation),shares border with,Hydrus
5,http://www.wikidata.org/entity/Q3304236,http://www.wikidata.org/entity/P131,http://www.wikidata.org/entity/Q9289,Lambda_Mensae,located in the administrative territorial entity,Mensa_(constellation)


In [12]:
get_uri_labels(doc_relation_df, entity_set, predicate_set_df)

,subject_uri,predicate_uri,object_uri,subject,predicate,object
0,http://www.wikidata.org/entity/Q3304236,http://www.wikidata.org/entity/P59,http://www.wikidata.org/entity/Q9289,Lambda_Mensae,constellation,Mensa_(constellation)
1,http://www.wikidata.org/entity/Q3304236,http://www.wikidata.org/entity/P31,http://www.wikidata.org/entity/Q523,Lambda_Mensae,instance of,Star
2,http://www.wikidata.org/entity/Q9289,http://www.wikidata.org/entity/P61,http://www.wikidata.org/entity/Q202703,Mensa_(constellation),discoverer or inventor,Nicolas-Louis_de_Lacaille
3,http://www.wikidata.org/entity/Q9289,http://www.wikidata.org/entity/P47,http://www.wikidata.org/entity/Q10457,Mensa_(constellation),shares border with,Chamaeleon
4,http://www.wikidata.org/entity/Q9289,http://www.wikidata.org/entity/P138,http://www.wikidata.org/entity/Q213360,Mensa_(constellation),named after,Table_Mountain
5,http://www.wikidata.org/entity/Q9289,http://www.wikidata.org/entity/P47,http://www.wikidata.org/entity/Q10416,Mensa_(constellation),shares border with,Hydrus


In [13]:
get_uri_labels(correct_relation_df, entity_set, predicate_set_df)

,subject_uri,predicate_uri,object_uri,subject,predicate,object
0,http://www.wikidata.org/entity/Q9289,http://www.wikidata.org/entity/P47,http://www.wikidata.org/entity/Q10457,Mensa_(constellation),shares border with,Chamaeleon
1,http://www.wikidata.org/entity/Q9289,http://www.wikidata.org/entity/P47,http://www.wikidata.org/entity/Q10416,Mensa_(constellation),shares border with,Hydrus


In [15]:
import pandas as pd
from helper_tools.evaluation import generate_pr_f1_score

(
    correct_relations, gold_standard_relations, predicted_relations,
    extracted_subjects, gold_standard_subjects, correct_extracted_subjects,
    extracted_predicates, gold_standard_predicates, correct_extracted_predicates,
    extracted_objects, gold_standard_objects, correct_extracted_objects,
    extracted_entities, gold_standard_entities, correct_extracted_entities
) = evaluate_doc(turtle_string, doc_id, relation_df)

# Hilfsfunktion zur Umwandlung in dict
def get_scores(correct, gold, predicted):
    precision, recall, f1 = generate_pr_f1_score(correct, gold, predicted)
    return {"Precision": precision, "Recall": recall, "F1-Score": f1}

# Ergebnisse sammeln
metrics = {
    "Relation": get_scores(correct_relations, gold_standard_relations, predicted_relations),
    "Subject": get_scores(correct_extracted_subjects, gold_standard_subjects, extracted_subjects),
    "Predicate": get_scores(correct_extracted_predicates, gold_standard_predicates, extracted_predicates),
    "Object": get_scores(correct_extracted_objects, gold_standard_objects, extracted_objects),
    "Entity": get_scores(correct_extracted_entities, gold_standard_entities, extracted_entities),
}

# DataFrame erzeugen
metrics_df = pd.DataFrame.from_dict(metrics, orient="index")
metrics_df

,Precision,Recall,F1-Score
Relation,0.333333,0.333333,0.333333
Subject,0.666667,1.000000,0.800000
Predicate,0.750000,0.600000,0.666667
Object,0.833333,0.833333,0.833333
Entity,0.857143,0.857143,0.857143


# Evaluation on Test

In [16]:
evaluation_log = []

In [17]:
for i in range(4):
    target_doc = docs.iloc[i]
    doc_id = target_doc["docid"]
    text = target_doc["text"]
    print(f"doc: {doc_id} - text: {text}")
    response = graph.invoke({"text": text, "messages":[], "debug": False}, config={"callbacks": [langfuse_handler], "recursion_limit": 100})
    evaluation_log.append([*evaluate_doc(re.search(r'<ttl>(.*?)</ttl>', response["messages"][-1], re.DOTALL).group(1), doc_id, relation_df)])

doc: 0 - text: Corfe Castle railway station is a station on the Swanage Railway in the village of Corfe Castle, in the United Kingdom.
doc: 1 - text: Ricardo Lumengo is a Swiss politician. He was born in Fribourg and lives in Biel/Bienne. He works in Bern and speaks the Kongo language.
doc: 2 - text: The National Parks Project is a nature documentary film presented in the ImagineNATIVE Film and Media Arts Festival. It is in Inuktitut.
doc: 3 - text: Lambda Mensae is a star in the constellation Mensa. It was discovered by Nicolas-Louis de Lacaille, and named after Table Mountain in South Africa. Mensa shares borders with Chamaeleon and Hydrus.


In [18]:
evaluation_log_df = pd.DataFrame(
    evaluation_log,
    columns=[
        "Correct Relations", "Gold Standard", "Total Predicted",
        "Extracted Subjects", "Gold Standard Subjects", "Correct Extracted Subjects",
        "Extracted Predicates", "Gold Standard Predicates", "Correct Extracted Predicates",
        "Extracted Objects", "Gold Standard Objects", "Correct Extracted Objects",
        "Extracted Entities", "Gold Standard Entities", "Correct Extracted Entities"
    ]
)
evaluation_log_df

,Correct Relations,Gold Standard,Total Predicted,Extracted Subjects,Gold Standard Subjects,Correct Extracted Subjects,Extracted Predicates,Gold Standard Predicates,Correct Extracted Predicates,Extracted Objects,Gold Standard Objects,Correct Extracted Objects,Extracted Entities,Gold Standard Entities,Correct Extracted Entities
0,1,4,5,3,1,1,3,4,2,3,4,3,4,5,4
1,4,5,6,1,1,1,6,5,4,6,5,5,7,6,6
2,1,4,3,1,1,1,3,4,1,3,4,3,4,5,4
3,2,6,6,3,2,2,4,5,3,6,6,5,7,7,6


In [20]:
import pandas as pd
from helper_tools.evaluation import generate_pr_f1_score

# Mapping der Spalten zu den jeweiligen Metriken
metric_map = {
    "Relation": ["Correct Relations", "Gold Standard", "Total Predicted"],
    "Subject": ["Correct Extracted Subjects", "Gold Standard Subjects", "Extracted Subjects"],
    "Predicate": ["Correct Extracted Predicates", "Gold Standard Predicates", "Extracted Predicates"],
    "Object": ["Correct Extracted Objects", "Gold Standard Objects", "Extracted Objects"],
    "Entity": ["Correct Extracted Entities", "Gold Standard Entities", "Extracted Entities"]
}

# Dictionaries für das Aufsummieren der Scores
metric_scores = {metric: {"precision": [], "recall": [], "f1": []} for metric in metric_map}

# Über alle Dokumente iterieren und Einzelwerte sammeln
for _, row in evaluation_log_df.iterrows():
    for metric, (correct_col, gold_col, pred_col) in metric_map.items():
        precision, recall, f1 = generate_pr_f1_score(
            row[correct_col], row[gold_col], row[pred_col]
        )
        metric_scores[metric]["precision"].append(precision)
        metric_scores[metric]["recall"].append(recall)
        metric_scores[metric]["f1"].append(f1)

# Macro Average berechnen
macro_scores = {
    metric: {
        "Precision": sum(scores["precision"]) / len(scores["precision"]) if scores["precision"] else 0.0,
        "Recall":    sum(scores["recall"])    / len(scores["recall"])    if scores["recall"]    else 0.0,
        "F1-Score":  sum(scores["f1"])       / len(scores["f1"])        if scores["f1"]       else 0.0
    }
    for metric, scores in metric_scores.items()
}

# DataFrame erstellen
macro_scores_df = pd.DataFrame.from_dict(macro_scores, orient="index")

# Ausgabe (optional)
macro_scores_df

,Precision,Recall,F1-Score
Relation,0.383333,0.408333,0.392136
Subject,0.750000,1.000000,0.825000
Predicate,0.604167,0.537500,0.562771
Object,0.916667,0.833333,0.864177
Entity,0.928571,0.864286,0.889499
